In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading Data
The data is in total about 18GB in size.Hence, it is practically impossible to load the entire data into memory as we will be getting a memory crash error and session will be restarted again and again. Hence, I made use of the Parquet files which Rob Mulla made available over here:https://www.kaggle.com/robikscube/ubiquant-parquet.

# Understanding Parquet Format

Parquet is an open source file format available to any project in the Hadoop ecosystem. It stores each column in a separate file as opposed to common row based storage formats like CSV. This approach is particularly helpful when we want to retrieve only a particular columns from the data, hence parquet format is optimized for such queries to greatly reduce the IO.

Inorder to read more about Parquet format, refer this link:https://databricks.com/glossary/what-is-parquet

Also, for some great starter info, refer this notebook by Rob Mulla : https://www.kaggle.com/robikscube/fast-data-loading-and-low-mem-with-parquet-files/notebook

In [ ]:
%%time
train_df = pd.read_parquet('../input/ubiquant-parquet/train.parquet')

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.dtypes

As we can see from above, there are 3114410 rows and 304 columns, out of which, the features f_0 to f_299 are anonymized features that have been generated using market data and all those are float64 format data, whereas time_id and investment_id are int64 format and row_id is object type which is nothing but row number.

Let's look at the number of unique_values in row_id, time_id, investment_id columns below

In [ ]:
discrete_cols = ["row_id", "time_id", "investment_id"]
for col in discrete_cols:
    print(col, " : ",train_df[col].nunique())

In [ ]:
first_part = []
second_part = []
for row_id in train_df["row_id"].values:
    if row_id.split("_")[0] not in first_part:
        first_part.append(row_id.split("_")[0])
    if row_id.split("_")[1] not in second_part:
        second_part.append(row_id.split("_")[1])

In [ ]:
len(first_part), len(second_part)

As we can see, the row_id is nothing but a combination of time_id and investment_id, with each time_id containing <= investment_id number of entries. So, each of these investment_id data is already present in separate parquet files when we load the data that Rob Mulla Provided, so we can make use of those files to inspect any Individual investment_id. So, let's see if we can find any patterns by looking at first investment_id data. It can be loaded as shown below:

In [ ]:
investment_id_df = pd.read_parquet("../input/ubiquant-parquet/investment_ids/1.parquet")

In [ ]:
investment_id_df.head()

In [ ]:
investment_id_df.shape

In [ ]:
investment_id_df.time_id.nunique()

Let's now look at some of the continuous_features and it's trend

In [ ]:
cont_cols = [f"f_{i}" for i in range(10)]
fig, axes = plt.subplots(5, 2, figsize=(20,20))
i=0
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)
for col in cont_cols:
    row = i//2
    column = i%2
    sns.kdeplot(data = investment_id_df, x=col, ax=axes[row, column], shade=True)
    i+=1
fig.show()

From the above plots, my observations are that:-

$f_1$, $f_2$, $f_5$, $f_7$ - close to normal distribution

$f_8$, $f_0$ - left tailed distribution 

$f_3$ - right tailed distribution 

$f_9$, $f_6$, $f_4$ - mixture of distributions

Now, let's look at target distribution

In [ ]:
sns.kdeplot(investment_id_df["target"], shade=True)

The target seems to follow normal distribution for this particular investment_id.

# Further Analysis will be continued. If you like my work, please consider to Upvote. Also, any suggestions or feedback would be really helpful